# Définition de méthodes et classes
## Exercice 1 : Création d’une méthode

In [1]:
class Point(object):
    def __init__(self, x=0, y=0):
        self.x, self.y = x, y

    def __str__(self):
        return 'Les coordonnées du point sont \nx = {}\ny = {}'.format(
            self.x, self.y)

    def distance(self):
        return abs(self.x - self.y)

In [2]:
P = Point(10, 50)
P.distance()

40

## Exercice 2 : Création d’une classe
Définir une classe « Fraction » représentant les fractions rationnelles réduites.

In [3]:
class Fraction():
    def __init__(self, x=1, y=1):
        if y == 0: raise ZeroDivisionError
        self.x, self.y = x, y
        self.num, self.den = abs(x), abs(y)
        self.quotient = x / y

        if x * y >= 0:
            self.signe = 1
        else:
            self.signe = -1

    def __str__(self):
        return "({}/{})".format(self.x, self.y)

    def __neg__(self):
        return self.signe * (self.quotient)

    def __add__(self, other):
        return self.quotient + other.quotient

    def __sub__(self, other):
        return self.quotient - other.quotient

    def __mul__(self, other):
        return self.quotient * other.quotient

    def simpliFrac(self):
        num, den = self.x, self.y
        div = 2
        while div <= min(num, den):
            if num % div == 0 and den % div == 0:
                num //= div
                den //= div
            else:
                div += 1
        self.x, self.y = num, den

In [4]:
F = Fraction(1, 10)
E = Fraction(60, 36)

In [5]:
E.simpliFrac()
print(E)

(5/3)


# Héritage surcharge et polymorphisme
## Exercice 3 : Compte bancaire simple
Nous nous intéressons à un compte simple caractérisé par un solde exprimé en Euros qui peut être positif ou négatif.

In [6]:
class CompteSimple():
    def __init__(self, solde, iban=""):
        self.solde = solde
        self.iban = iban

    def __str__(self):
        return "Le solde du compte {} est de {} €".format(
            self.iban, self.solde)

    def enregistrerOperation(self, operation):
        self.solde += operation

In [7]:
livret = CompteSimple(10)
print(livret)
livret.enregistrerOperation(-5)
print(livret)

Le solde du compte  est de 10 €
Le solde du compte  est de 5 €


## Exercice 4 : Compte courant
Une banque conserve pour chaque compte l’historique des opérations qui le concernent (on se limite ici aux opérations de crédits et de débits). On souhaite modéliser un tel compte qu’on appelle compte courant.

In [8]:
class CompteCourant(CompteSimple):
    def __init__(self, solde, iban=""):
        CompteSimple.__init__(self, solde)
        self.historique = []
        self.iban = iban

    def enregistrerOperation(self, operation):
        self.historique.append(operation)
        self.solde += operation

    def afficherReleve(self):
        for operation in self.historique:
            if operation > 0: print("+" + str(operation))
            else: print(str(operation))

    def afficherReleveCredits(self):
        for operation in self.historique:
            if operation > 0: print("+" + str(operation))

    def afficherReleveDebits(self):
        for operation in self.historique:
            if operation < 0: print(str(operation))

In [9]:
cc = CompteCourant(1200)
print(cc)
cc.enregistrerOperation(-500)
cc.enregistrerOperation(-300)
cc.enregistrerOperation(-9.99)
cc.enregistrerOperation(+800)
cc.afficherReleve()
print(cc)

Le solde du compte  est de 1200 €
-500
-300
-9.99
+800
Le solde du compte  est de 1190.01 €


## Exercice 5 : La banque
Une banque est un organisme qui gère un grand nombre de comptes, qu’ils soient simples ou courants.

In [10]:
class Banque():
    def __init__(self):
        self.comptes = []

    def openCS(self, solde):
        self.comptes.append(CompteSimple(solde))

    def openCC(self, solde):
        self.comptes.append(CompteCourant(solde))

    def total(self):
        return sum(compte.solde for compte in self.comptes)

    def frais(self, montant=-2):
        for compte in self.comptes:
            compte.enregistrerOperation(montant)

    def __str__(self):
        chaine = ""
        for compte in self.comptes:
            chaine += str(compte) + "\n"
        return chaine

    def releveCC(self):
        for compte in self.comptes:
            if isinstance(compte, CompteCourant):
                print(compte)
                compte.afficherReleve()

    def ajoutCompte(self, compte):
        """
        Ajouter un compte externe à la banque
        """
        if issubclass(type(compte), CompteSimple):
            self.comptes.append(compte)

In [11]:
b = Banque()
b.openCS(10)
b.openCS(200)
b.openCC(500)
b.openCC(5)
b.ajoutCompte(livret)
b.ajoutCompte(cc)
print(b.total())
b.frais()
print(b.total())
b.releveCC()

1910.01
1898.01
Le solde du compte  est de 498 €
-2
Le solde du compte  est de 3 €
-2
Le solde du compte  est de 1188.01 €
-500
-300
-9.99
+800
-2


## Exercice 6 : Numéro de compte
Chaque compte en France doit être identifié par un numéro IBAN dont le format est le suivant : **FRkk BBBB BGGG GGCC CCCC CCCC CKK**
- **FRkk** = code pays ;
- **B** = code banque ;
- **G** = code guichet ;
- **C** = numéro de compte ;
- **K** = clef.

Dans notre classe « Banque » on prendra :
- « FR76 » pour le code pays ;
- « 12345 » pour le code banque ;
- « 00001 » pour le code guichet (On suppose que notre banque n’a qu’une seule agence) ;
- Un numéro incrémenté de 9 chiffres pour chaque numéro de compte auquel on ajoute à la fin un code de 2 chiffres pour le type de compte :
- Compte simple : Commence par « 000000001 » et on concatène « 60 » à la fin ;
- Compte courant : Commence par « 000000001 » et on concatène « 40 » à la fin.
- La clé est une valeur calculée par la fonction : $$KK=(CC CCCC CCCC C) \mod 2$$ *(La fonction pour calculer la clé est volontairement très simple pour ne pas s’attarder sur cepoint)*

In [12]:
class BanqueFR(Banque):
    def __init__(self, pays="76", banque="12345", guichet="00001"):
        Banque.__init__(self)
        self.comptes = []
        self.pays = "FR" + pays
        self.banque = banque
        self.guichet = guichet
        self.nbCompte = 0

    def openCS(self, solde):
        self.nbCompte += 1
        kk = str(self.nbCompte % 2).zfill(2)
        iban = self.pays + self.banque + self.guichet + str(
            self.nbCompte).zfill(9) + "60" + kk
        self.comptes.append(CompteSimple(solde, iban))

    def openCC(self, solde):
        self.nbCompte += 1
        kk = str(self.nbCompte % 2).zfill(2)
        iban = self.pays + self.banque + self.guichet + str(
            self.nbCompte).zfill(9) + "40" + kk
        self.comptes.append(CompteCourant(solde, iban))

In [13]:
bf = BanqueFR()
bf.openCS(100)
bf.openCC(5)
bf.openCS(1.50)
bf.openCC(1500)
print(bf)

Le solde du compte FR7612345000010000000016001 est de 100 €
Le solde du compte FR7612345000010000000024000 est de 5 €
Le solde du compte FR7612345000010000000036001 est de 1.5 €
Le solde du compte FR7612345000010000000044000 est de 1500 €

